# Persistent homology for smartphone data analysis (pedestrian recognition)

__Description__: The goal of this is project to illustrate, on a toy example, the benefit of “coordinate in- variance” of persistent homology. The walk of 3 pedestrians A, B and C, has been recorded using the accelerometer sensor of a smartphone carried in the their pocket, giving rise to 3 multivariate time series in R3: each time series represents the 3 coordinates of the acceleration of the corresponding pedestrian in a coordinate system attached to the sensor. As, the smartphone was carried in unknown different positions and was not fixed, these time series cannot be compared coordinates by coordinates. Using a sliding window, each series has been splitted in a list of 100 times series made of 200 consecutive points, that are stored in data A, data B and data C. To each set of 200 points is associated a label A, B or C stored in label (see the data set and the Python script to load the data). The objective is to compute the persistence diagrams of these 3D point clouds and use them to achieve a pedestrian recognition task (supervised setting).
Note: This project requires some (basic) knowledge of learning (random forests).

In [41]:
import numpy as np
import pickle as pickle
from os.path import join
import matplotlib.pyplot as plt
%matplotlib inline

## ## Question 1: Loading Data

In [42]:
path="/Users/yaguethiam/PersistentHomology/data_acc_rot.dat"

In [43]:
f = open(path,"rb")
data = pickle.load(f,encoding='latin1')
f.close()

data_A = data[0]
data_B = data[1] 
data_C = data[2]
label = data[3]
print(len(data_A[0]))

200


## Question 2:

Compute and save the 0-dimensional and 1-dimensional persistence diagrams of the Rips filtrations (or alternately the alpha-shape filtrations) built on top of each of the 300 point clouds in R3.

In [44]:
path_to_cython_gudhi='/Users/yaguethiam/2017-10-02-10-19-30_GUDHI_2.0.1/build/cython'
import sys
sys.path.append(path_to_cython_gudhi)
import gudhi

## About persistent homology 

### General definitions
Let (P, D) be a metric space where P is a point set. 
Given r > 0, the Rips complex is the simplicial complex R(P) constituted by the simplexes such that$d(p,q) \leq r$ for every pair of vertices in the simplex.  
Constructing the Rips complex helps capture the topology of the data set. Choosing $r$ is a difficult task because if $r$ ois too small, the complex is a discrete set, and if $r$ is too large, the complex becomes a single high-dimensional complex.


Given a filtration $(K_0, K_1, ..., K_n)$, the $p$-dimensional persistence diagram is the set of points (i,j) such that the number of $p$-dimensional homology classes born at $K_i$ that die entering $K_j$ is one.  
Here the filtration will actually be a sequence of Rips complexes associated to the 3D point cloud for an increasing
sequence of parameter values ($r_i$).


### 0-dimensional persistence diagrams of the Rips filtrations

In order to draw the 0-dimensional persistence diagram of the Rips filtrations, we use \verb gudhi.RipsComplex on the 3D point cloud. 

> __parameter to tune is the maxEdgeLen__

In [45]:
def build_0_persistent_diag(dataset, maxEdgeLen=10000):
    persist=[]
    for i in range(len(dataset)):
        rips = gudhi.RipsComplex(points=dataset[i],max_edge_length=maxEdgeLen)
        simplex_tree = rips.create_simplex_tree(max_dimension=1)
        diag = simplex_tree.persistence()
        persist.append(diag)
    return persist

> Compute the 0 persistent diagram for the 3 pedestrians 

In [46]:
diagA_0=build_0_persistent_diag(data_A)
diagB_0=build_0_persistent_diag(data_B)
diagC_0=build_0_persistent_diag(data_C)

### 1-dimensional persistence diagrams of the Rips filtrations

In [49]:
def build_1_persistent_diag(dataset, maxEdgeLen=100):
    persist=[]
    for i in range(len(dataset)):
        rips = gudhi.RipsComplex(points=dataset[i],max_edge_length=maxEdgeLen)
        simplex_tree = rips.create_simplex_tree(max_dimension=2)
        diag = simplex_tree.persistence()
        persist.append(diag)
    return persist

> Compute the 1 persistent diagram for the 3 pedestrians 

In [50]:
diagA_1=build_1_persistent_diag(data_A)
diagB_1=build_1_persistent_diag(data_B)
diagC_1=build_1_persistent_diag(data_C)

## Question 3: Matrices of pairwise bottleneck distances between diagrams and use a dimensionality reduction algorithm to visualize them in 2D and 3D (e.g. Multidimensional Scaling).


The bottleneck distance measures the similarity between two persistence diagrams. It is the shortest distance $d$ for which there exists a perfect matching between the points of the two diagrams such that any couple of matched points are at distance at most $d$. The cost of matching, i.e. taking a point $p$ of the first diagram to a point $p'$ of the second diagram corresponds the minimum between moving $p$ to $p'$ and moving both points on the diagonal.

USE Gudhi::persistence_diagram::bottleneck_distance (const Persistence_diagram1 &diag1, const Persistence_diagram2 &diag2, double e=(std::numeric_limits< double >::min)())

Why are we comparing 0 and 1-dimensional persistence diagrams of the Rips filtrations ? 

BLABLABLABLA

USE sklearn.manifold.MDS for multidimensional scaling. 

remarques du cours
→ Vietoris-Rips (or Cech, witness) filtrations quickly become prohibitively large as
the size of the data increases ( O(|X|
d
) ), making the computation of persistence
practically almost impossible.
→ Persistence diagrams of Rips-Vietoris (and Cˇech, witness,..) filtrations and
Gromov-Hausdorff distance are very sensitive to noise and outliers

In [34]:
from sklearn import manifold

In [80]:
def computeMatrixBottleneckDistance(diagList1):
    matrix=np.zeros((len(diagList1),len(diagList1)))
    for i in range(len(diagList1)):
        for j in range(i,len(diagList1)):
            dist=gudhi.bottleneck_distance(diagList1[i], diagList1[j], 0.001)
            matrix[i,j]=dist
            matrix[j,i]=dist
    return matrix

In [ ]:
matrix_bottleneck_distance_A_0=computeMatrixBottleneckDistance(diagA_0)
matrix_bottleneck_distance_A_1=computeMatrixBottleneckDistance(diagA_1)

matrix_bottleneck_distance_B_0=computeMatrixBottleneckDistance(diagB_0)
matrix_bottleneck_distance_B_1=computeMatrixBottleneckDistance(diagB_1)

matrix_bottleneck_distance_C_0=computeMatrixBottleneckDistance(diagC_0)
matrix_bottleneck_distance_C_1=computeMatrixBottleneckDistance(diagC_1)

> ## With 0-persistent homology

### Pedestrian A

In [ ]:
mds2 = manifold.MDS(n_components=2, max_iter=3000, eps=1e-9,
                   dissimilarity="precomputed", n_jobs=1)
pos = mds2.fit(matrix_bottleneck_distance_A_0).embedding_

plt.subplots_adjust(bottom = 0.1)
plt.scatter(
    pos[:, 0], pos[:, 1], marker = 'o'
    )
plt.show()

### Pedestrian B

In [ ]:
mds2 = manifold.MDS(n_components=2, max_iter=3000, eps=1e-9,
                   dissimilarity="precomputed", n_jobs=1)
pos = mds2.fit(matrix_bottleneck_distance_B_0).embedding_

plt.subplots_adjust(bottom = 0.1)
plt.scatter(
    pos[:, 0], pos[:, 1], marker = 'o'
    )
plt.show()

### Pedestrian C

In [ ]:
mds2 = manifold.MDS(n_components=2, max_iter=3000, eps=1e-9,
                   dissimilarity="precomputed", n_jobs=1)
pos = mds2.fit(matrix_bottleneck_distance_C_0).embedding_

plt.subplots_adjust(bottom = 0.1)
plt.scatter(
    pos[:, 0], pos[:, 1], marker = 'o'
    )
plt.show()

> ## With 0-persistent homology

### Pedestrian A

In [ ]:
mds2 = manifold.MDS(n_components=2, max_iter=3000, eps=1e-9,
                   dissimilarity="precomputed", n_jobs=1)
pos = mds2.fit(matrix_bottleneck_distance_A_1).embedding_

plt.subplots_adjust(bottom = 0.1)
plt.scatter(
    pos[:, 0], pos[:, 1], marker = 'o'
    )
plt.show()

### Pedestrian B

In [ ]:
mds2 = manifold.MDS(n_components=2, max_iter=3000, eps=1e-9,
                   dissimilarity="precomputed", n_jobs=1)
pos = mds2.fit(matrix_bottleneck_distance_B_1).embedding_

plt.subplots_adjust(bottom = 0.1)
plt.scatter(
    pos[:, 0], pos[:, 1], marker = 'o'
    )
plt.show()

### Pedestrian C

In [ ]:
mds2 = manifold.MDS(n_components=2, max_iter=3000, eps=1e-9,
                   dissimilarity="precomputed", n_jobs=1)
pos = mds2.fit(matrix_bottleneck_distance_C_1).embedding_

plt.subplots_adjust(bottom = 0.1)
plt.scatter(
    pos[:, 0], pos[:, 1], marker = 'o'
    )
plt.show()

## Question 4: Computing persistence landscape
This function should take as input a persistence
diagram dgm (in the Gudhi format), a dimension k, the endpoints xmin, xmax of an interval, the
number nb_nodes of nodes of a regular grid on the interval [xmin, xmax] and a number of landscapes
nb_ld, and output a nbld × nbnodes array storing the values of the first nb_ld landscapes of dgm on the
node of the grid. Check, on some simple examples that your code is correct.

In [35]:
def computeLandscapes(dgm,x_min,x_max,nb_nodes,nb_lbd):
    ''' This function compute the approximate landscape of a persistent diagram. 
    Given a range [xmin,xmax] and a grid size '''
    
    output = np.zeros((nb_lbd,nb_nodes))
    step = (x_max - x_min) / nb_nodes
    for i in range(nb_nodes):
        x = x_min + i * step
        event_list = []
        for tup in dgm:
            b = tup[0]
            d = tup[1]
            if (b <= x) and (x<= d):
                if x >= (d+b)/2. :
                    event_list.append((d-x))
                else:
                    event_list.append((x-b))
        event_list.sort(reverse=True)
        event_list = np.asarray(event_list)
        for j in range(nb_lbd):
            if(j<len(event_list)):
                output[j,i]=event_list[j]
    return output

> Example

In [36]:
diagA_1_pair_test= [diagA_1[0][a][1] for a in range(len(diagA_1[0]))] 
computeLandscapes(diagA_1_pair_test,0,15,400,5)

array([[ 0.    ,  0.0375,  0.075 , ..., 14.8875, 14.925 , 14.9625],
       [ 0.    ,  0.0375,  0.075 , ...,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.0375,  0.075 , ...,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.0375,  0.075 , ...,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.0375,  0.075 , ...,  0.    ,  0.    ,  0.    ]])

#### Prepare pair of (death,birth) for persistence landscape computation

In [ ]:
def buildPair(diagram):
    return [item[1] for item in diagram] 

In [12]:
diagA_0_pairs=[buildPair(diag) for diag in diagA_0] 
diagA_1_pairs=[buildPair(diag) for diag in diagA_1] 

diagB_0_pairs=[buildPair(diag) for diag in diagB_0] 
diagB_1_pairs=[buildPair(diag) for diag in diagB_1] 

diagC_0_pairs=[buildPair(diag) for diag in diagC_0] 
diagC_1_pairs=[buildPair(diag) for diag in diagC_1] 

>  Choose parameters

In [14]:
x_min=0
x_max=5
nb_nodes=300
nb_lbd=5

In [15]:
diagA_0_landscape=computeLandscapes(diagA_0_pair,x_min,x_max,nb_nodes,nb_lbd)
diagA_1_landscape=computeLandscapes(diagA_1_pair,x_min,x_max,nb_nodes,nb_lbd)

diagB_0_landscape=computeLandscapes(diagB_0_pair,x_min,x_max,nb_nodes,nb_lbd)
diagB_1_landscape=computeLandscapes(diagA_1_pair,x_min,x_max,nb_nodes,nb_lbd)

diagA_0_landscape=computeLandscapes(diagA_0_pair,x_min,x_max,nb_nodes,nb_lbd)
diagA_1_landscape=computeLandscapes(diagA_1_pair,x_min,x_max,nb_nodes,nb_lbd)

### For each 0-dimensional and 1-dimensional persistence diagrams, compute the first 5 landscapes on a
relevant interval with a few hundred of nodes. Splitting randomly the data set into a 80/20 learning/test
data, use a random forest to explore the performances of the 0-dimensional or 1-dimensional landscapes
to classify pedestrians. An example of code to realize such an experiment can be downloaded at http:
//geometrica.saclay.inria.fr/team/Fred.Chazal/Centrale2017.html. Compare the results you
obtain using 0-dimensional landscapes, 1-dimensional landscapes or both.



### Do the same experiment as previously, but using the raw data ( 3 × 200 array of acceleration coordinates).
Compare the obtained classification results to the previous one.

## Random Forest

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sun Dec 17 18:33:31 2017

@author: Fredreci Chazal - All rights reserved
"""
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

#### Classification with random forests 
#### Interesting compare with L0_list, L1_list and L_list
#### where L0_list, L1_list and L_list are list storing the $0$-dimensional 
#### landscapes, $1$-dimensional landscapes, and the concatenation of both 
#### respectively
avg = 0
for i in range(20):
    L_train, L_test, label_train, label_test = train_test_split(L1_list, label, test_size=0.2)
    RF = RandomForestClassifier()
    RF.fit(L_train, label_train)
    print(np.mean(RF.predict(L_test) == label_test) )
    avg += np.mean(RF.predict(L_test) == label_test)
    #print(confusion_matrix(RF.predict(L_test), label_test))
print ("avg pred: ",avg/20)

plt.plot(RF.feature_importances_)